In [5]:
import os

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy import asarray
from numpy.random import randn
from numpy.random import randint

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import load_img

from matplotlib import pyplot
from matplotlib import pyplot as plt

from keras.models import load_model
from tqdm import tqdm
import numpy as np
import PIL
from PIL import Image

In [2]:
def define_discriminator(input_shape=(128,128,3), n_classes=3):
    # Getting the label input
    label_input = Input(shape=(1,))
    # embedding the label and reshaping it
    embedded_label = Embedding(n_classes, 50)(label_input)
    n_nodes = input_shape[0] * input_shape[1]
    embedded_label = Dense(n_nodes)(embedded_label)
    embedded_label = Reshape((input_shape[0], input_shape[1], 1))(embedded_label)
    
    # Getting the image input
    image_input = Input(shape=input_shape)
    # concat label to image
    labelled_image = Concatenate()([image_input, embedded_label])
    
    # layer 1
    layer_1 = Conv2D(128, (3,3), strides=(2,2), padding='same')(labelled_image)
    layer_1 = LeakyReLU(alpha=0.2)(layer_1)
    # layer 2
    layer_2 = Conv2D(128, (3,3), strides=(2,2), padding='same')(layer_1)
    layer_2 = LeakyReLU(alpha=0.2)(layer_2)
    # layer 3
    layer_3 = Conv2D(128, (3,3), strides=(2,2), padding='same')(layer_2)
    layer_3 = LeakyReLU(alpha=0.2)(layer_3)
    # layer 4
    layer_4 = Conv2D(128, (3,3), strides=(2,2), padding='same')(layer_3)
    layer_4 = LeakyReLU(alpha=0.2)(layer_4)
    # layer 5
    layer_5 = Conv2D(128, (3,3), strides=(2,2), padding='same')(layer_4)
    layer_5 = LeakyReLU(alpha=0.2)(layer_5)
    
    flatten_layer = Flatten()(layer_5)
    dropout_layer = Dropout(0.4)(flatten_layer)
    output_layer = Dense(1, activation='sigmoid')(dropout_layer)
    # defining the model
    model = Model([image_input, label_input], output_layer)
    # compiling the model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    print(model.summary())
    return model


In [3]:
def define_generator(latent_space, n_classes=3):
    #Getting the label input
    label_input = Input(shape=(1,))
    # embedding the label input and reshaping it
    embedded_layer = Embedding(n_classes, 50)(label_input)
    n_nodes = 4 * 4
    embedded_layer = Dense(n_nodes)(embedded_layer)
    embedded_layer = Reshape((4, 4, 1))(embedded_layer)
    # Getting the latent input
    latent_input = Input(shape=(latent_space,))
    # creating a 4*4 space
    n_nodes = 256 * 4 * 4
    # Creating the base image
    base_image = Dense(n_nodes)(latent_input)
    base_image = LeakyReLU(alpha=0.2)(base_image)
    base_image = Reshape((4, 4, 256))(base_image)
    # concat the base image and label input
    labelled_base_image = Concatenate()([base_image, label_input])
    
    # upsampling to 8x8 image
    layer_1 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(labelled_base_image)
    layer_1 = LeakyReLU(alpha=0.2)(layer_1)
    # upsampling to 16x16 image
    layer_2 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(layer_1)
    layer_2 = LeakyReLU(alpha=0.2)(layer_2)
    # upsampling to 32x32 image
    layer_3 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(layer_2)
    layer_3 = LeakyReLU(alpha=0.2)(layer_3)
    # upsampling to 64x64 image
    layer_4 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(layer_3)
    layer_4 = LeakyReLU(alpha=0.2)(layer_4)
    #upsampling to 128x128 image
    layer_5 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(layer_4)
    layer_5 = LeakyReLU(alpha=0.2)(layer_5)
    # output layer with tanh function
    output_layer = Conv2D(3, (3, 3), activation='tanh', padding='same')(layer_5)
    # defining the model
    model = Model([latent_input, label_input], output_layer)
    print(model.summary())
    return model
    

In [4]:
def define_gan(generator, discriminator):
    # making the weights untrainable
    discriminator.trainable = False
    # getting the Generator noise and label inputs
    gen_noise, gen_label = generator.input
    # getting the Generator output image
    gen_output = generator.output
    # Getting the GAN output
    # Inputting Generator output and Generator label
    gan_output = discriminator([gen_output, gen_label])
    # defining the GAN model
    model = Model([gen_noise, gen_label], gan_output)

    # compiling the model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
def load_real_samples():

    data_dir = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\bg-rm-images\\all_augmented4_and_original\\'
    #loading the images and their labels
    batch_size = 1
    img_height = 128
    img_width = 128
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.1,
        subset="training",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)
    image_list = list()
    label_list = list()
    for image_batch, labels_batch in train_ds:
        images = np.asarray(image_batch)
        images = images.astype('float32')
        images = (images - 127.5) / 127.5
        image_list.append(images[0])
        label_list.append(labels_batch.numpy())
    image_arr = np.array(image_list)
    label_arr = np.array(label_list)
    return [image_arr, label_arr]


In [6]:
def generate_real_samples(dataset, n_samples):
    # spliting the datset into images and labels
    images, labels = dataset
    # selecting random image + label based on number of samples
    ix = randint(0, images.shape[0], n_samples)
    X, labels = images[ix], labels[ix]
    # labeling these samples as real
    y = ones((n_samples, 1))
    return [X, labels], y

In [7]:
def generate_latent_points(latent_space, n_samples, n_classes=3):
    # generating points in the latent space
    x_input = randn(latent_space * n_samples)
    # reshaping into network inputs
    z_input = x_input.reshape(n_samples, latent_space)
    # generating latent point labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]
 
def generate_fake_samples(generator, latent_space, n_samples):
    # generating points in latent space
    z_input, labels_input = generate_latent_points(latent_space, n_samples)
    # creating fake images
    images = generator.predict([z_input, labels_input])
    # labeling these samples as fake
    y = zeros((n_samples, 1))
    return [images, labels_input], y

In [8]:
def save_plot(examples, epoch, n=7):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i])
    # save plot to file
    filename = 'generated_plot_v5t1_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

In [9]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_space, n_samples=150):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_space, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake[0], epoch)
    # save the generator model tile file
    filename = 'og_augmented_generator_model_v5t1_%03d.h5' % (epoch+1)
    g_model.save(filename)

In [10]:
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
    # plot loss
    pyplot.subplot(2, 1, 1)
    pyplot.plot(d1_hist, label='d-real')
    pyplot.plot(d2_hist, label='d-fake')
    pyplot.plot(g_hist, label='gen')
    pyplot.legend()
    # plot discriminator accuracy
    pyplot.subplot(2, 1, 2)
    pyplot.plot(a1_hist, label='acc-real')
    pyplot.plot(a2_hist, label='acc-fake')
    pyplot.legend()
    # save plot to file
    pyplot.savefig('plot_line_plot_loss_v5t1.png')
    pyplot.close()

In [11]:
def train(generator, discriminator, gan_model, dataset, latent_space, n_epochs=100, n_batch=8):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    n_steps = bat_per_epo * n_epochs
    d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            # getting random real samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            # updating discriminator model weights
            d_loss1, d_acc1 = discriminator.train_on_batch([X_real, labels_real], y_real)
            # generating fake samples
            [X_fake, labels], y_fake = generate_fake_samples(generator, latent_space, half_batch)
            # updating discriminator model weights
            d_loss2, d_acc2 = discriminator.train_on_batch([X_fake, labels], y_fake)
            # preparing latent input for the generator
            [z_input, labels_input] = generate_latent_points(latent_space, n_batch)
            # creating inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # updating the generator via the discriminator's error
            g_loss = generator.train_on_batch([z_input, labels_input], y_gan)
            # summarizing batch loss
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
            d1_hist.append(d_loss1)
            d2_hist.append(d_loss2)
            g_hist.append(g_loss)
            a1_hist.append(d_acc1)
            a2_hist.append(d_acc2)
        # evaluate the model performance
        if (i+1) % 10 == 0:
            summarize_performance(i, generator, discriminator, dataset, latent_space)
    plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)

In [ ]:
# defining the size of the latent space
latent_space = 100
# defining the discriminator
discriminator = define_discriminator()
# defining the generator
generator = define_generator(latent_space)
# defining the gan
gan_model = define_gan(generator, discriminator)
# loading image dataset
dataset = load_real_samples()
# training the model
train(generator, discriminator, gan_model, dataset, latent_space)

In [ ]:
# generating points in latent space as input for the generator
def generate_latent_points(latent_space, n_samples, n_classes=3):
    # generating points in the latent space
    x_input = randn(latent_space * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_space)
    # generating labels
    labels = randint(0, n_classes, n_samples)
    print("labels in gen function")
    print(labels)
    print(type(labels))
    return [z_input, labels]
 
# creating a plot of generated images
def save_plot(examples, n, n2):
    # plot images
    for i in range(n * n2):
        # define subplot
        pyplot.subplot(n, n2, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0])
    pyplot.show()

#loading the saved model
model = load_model('og_augmented_generator_model_v5t1_070.h5')
# generate images
underweight_latent_points, labels =  generate_latent_points(100, 100)
standardweight_latent_points, labels =  generate_latent_points(100, 100)
overweight_latent_points, labels =  generate_latent_points(100, 100)

# specify labels
labels = asarray([x for _ in range(50) for x in range(3)])
overweight_labels = np.full(100, 0)
standardweight_labels = np.full(100, 1)
underweight_labels = np.full(100, 2)


# generating underweight images
underweight  = model.predict([underweight_latent_points, underweight_labels])
# scale from [-1,1] to [0,1]
underweight = (underweight + 1) / 2.0
# plot the result
save_plot(underweight, 100, 1)

# generating standardweight images
standardweight  = model.predict([standardweight_latent_points, standardweight_labels])
# scale from [-1,1] to [0,1]
standardweight = (standardweight + 1) / 2.0
# plot the result
save_plot(standardweight, 100, 1)

# generating overweight images
overweight  = model.predict([overweight_latent_points, overweight_labels])
# scale from [-1,1] to [0,1]
overweight = (overweight + 1) / 2.0
# plot the result
save_plot(overweight, 100, 1)

In [ ]:

# saving synthetic underweight images
save_underweight_images_dir = 'C:\\Users\\Syed Johan Arif\\Desktop\\Workspace\\Practice\\Generated Images\\final_gan_v2\\underweight'
under_number = 101
for image in underweight:
    pyplot.imshow(image)
    pyplot.show()
    under_filename = save_underweight_images_dir + '\\cgan_image_' + str(under_number) + '.jpg'
    im = Image.fromarray((image * 255).astype(np.uint8))
    im = im.save(under_filename)
    under_number = under_number + 1

# saving synthetic standardweight images
save_standardweight_images_dir = 'C:\\Users\\Syed Johan Arif\\Desktop\\Workspace\\Practice\\Generated Images\\final_gan_v2\\standardweight'
standard_number = 101
for image in standardweight:
    pyplot.imshow(image)
    pyplot.show()
    standard_filename = save_standardweight_images_dir + '\\cgan_image_' + str(standard_number) + '.jpg'
    print(standard_filename)
    im = Image.fromarray((image * 255).astype(np.uint8))
    im = im.save(standard_filename)
    standard_number = standard_number + 1    

# saving synthetic overweight images
save_overweight_images_dir = 'C:\\Users\\Syed Johan Arif\\Desktop\\Workspace\\Practice\\Generated Images\\final_gan_v2\\overweight'
over_number = 101
for image in overweight:
    pyplot.imshow(image)
    pyplot.show()
    over_filename = save_overweight_images_dir + '\\cgan_image_' + str(over_number) + '.jpg'
    print(over_filename)
    im = Image.fromarray((image * 255).astype(np.uint8))
    im = im.save(over_filename)
    over_number = over_number + 1